# COS 710: Genetic Programming
## Assignment 1: COVID 19 Time Series

This assignment is conducted using the *gp-learn* package which extends *scikit-learn* for genetic programming in the domain of symbolic regression 

In [1]:
!pip install PyDrive
!pip install gplearn

     |████████████████████████████████| 51kB 2.0MB/s 


In [3]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [4]:
%pylab inline
from gplearn.genetic import SymbolicRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import graphviz

Populating the interactive namespace from numpy and matplotlib


In [172]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [169]:
drive.CreateFile({'id':"1XT6NWOJXF2sn-mFQ5iE16Hpl8H5m_s0F"}).GetContentFile("inputs.csv")
drive.CreateFile({'id':"15375p7kjGUrj5T-9vszE1-zDpvVV5Uw-"}).GetContentFile("targets_c.csv")
drive.CreateFile({'id':"1Ds0gi9Ya2C6-cjS-h-VD_JYOFhMaNPE9"}).GetContentFile("targets_d.csv")
drive.CreateFile({'id':"1c2dftnI74KnSB7EYXK0ZzU8KRM_fimc1"}).GetContentFile("targets_r.csv")


input_df = pd.read_csv("inputs.csv")
target_c_df = pd.read_csv("targets_c.csv")
target_d_df = pd.read_csv("targets_d.csv")
target_r_df = pd.read_csv("targets_r.csv")


InvalidConfigError: ignored

In [173]:
drive.CreateFile({'id':"1ZOtktxL3eErnRJuoS-b54a78D_LMnVgG"}).GetContentFile("difference_data.csv")
differences_df = pd.read_csv("difference_data.csv")


In [174]:
differences_df

,c_dif_1,c_dif_2,c_dif_3,c_dif_4,c_dif_5,c_dif_6,c_dif_target,r_dif_1,r_dif_2,r_dif_3,r_dif_4,r_dif_5,r_dif_6,r_dif_target,d_dif_1,d_dif_2,d_dif_3,d_dif_4,d_dif_5,d_dif_6,d_dif_target
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38911,75,97,82,104,85,47,70,96,307,71,49,45,1756,257,18,6,0,2,2,3,6
38912,97,82,104,85,47,70,265,307,71,49,45,1756,257,337,6,0,2,2,3,6,9
38913,82,104,85,47,70,265,102,71,49,45,1756,257,337,83,0,2,2,3,6,9,1
38914,104,85,47,70,265,102,70,49,45,1756,257,337,83,62,2,2,3,6,9,1,1


In [204]:
# Manipulating the data
from sklearn.model_selection import train_test_split
input_df
confirmed_only_df = input_df.drop(["r1", "r2", "r3", "r4", "r5", "r6", "r7", "d1", "d2", "d3", "d4", "d5", "d6", "d7"], axis=1)
recovered_only_df = input_df.drop(["c1", "c2", "c3", "c4", "c5", "c6", "c7", "d1", "d2", "d3", "d4", "d5", "d6", "d7"], axis=1)
deaths_only_df = input_df.drop(["r1", "r2", "r3", "r4", "r5", "r6", "r7", "c1", "c2", "c3", "c4", "c5", "c6", "c7"], axis=1)

# for difference data
difference_inputs_whole = differences_df.drop(["c_dif_target","d_dif_target","r_dif_target"], axis=1)
difference_inputs_c = differences_df.drop(["d_dif_target","r_dif_target", "r_dif_1","r_dif_2","r_dif_3","r_dif_4","r_dif_5","r_dif_6","d_dif_1","d_dif_2","d_dif_3","d_dif_4","d_dif_5","d_dif_6"], axis=1)
difference_inputs_r = differences_df.drop(["c_dif_target","d_dif_target", "c_dif_1","c_dif_2","c_dif_3","c_dif_4","c_dif_5","c_dif_6","d_dif_1","d_dif_2","d_dif_3","d_dif_4","d_dif_5","d_dif_6"], axis=1)
difference_inputs_d = differences_df.drop(["c_dif_target","r_dif_target", "r_dif_1","r_dif_2","r_dif_3","r_dif_4","r_dif_5","r_dif_6","c_dif_1","c_dif_2","c_dif_3","c_dif_4","c_dif_5","c_dif_6"], axis=1)

difference_c_train, difference_c_test = train_test_split(difference_inputs_c, test_size=0.35)
difference_r_train, difference_r_test = train_test_split(difference_inputs_r, test_size=0.35)
difference_d_train, difference_d_test = train_test_split(difference_inputs_d, test_size=0.35)



confirmed_only_df = confirmed_only_df.join(target_c_df)
confirmed_sample = confirmed_only_df.sample(n = 20000)
confirmed_sample["target_confirmed"]

whole_df_c = input_df.join(target_c_df)
whole_sample_c = whole_df_c.sample(n = 20000)

test = confirmed_only_df.sample(n = 10)
test


,c1,c2,c3,c4,c5,c6,c7,target_confirmed
30219,699,699,699,699,699,702,704,704
236,0,0,0,0,0,0,0,0
36256,0,0,0,0,0,0,0,0
26012,973,973,974,974,974,978,980,980
23240,3,3,12,14,28,36,42,48
33499,741,780,801,837,904,943,1001,1029
7848,276055,286020,295508,306181,317651,327850,334979,345714
31117,8,8,10,10,10,10,10,10
15401,40,41,41,43,44,47,57,57
25942,18,27,27,74,98,120,144,184


In [184]:
from gplearn.functions import make_function
import math

# tools
def cube(x):
  return x**3

def square(x):
  return x**2

def exponen(x):
  return ((math.e)**x)

cube = make_function(function=cube, name="cube", arity=1)
square = make_function(function=square, name="square", arity=1)
exponen = make_function(function=exponen, name="exponen", arity=1)

functions = ["add", "sub", "mul", "div", "inv", "log"]
features = ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "r1", "r2", "r3", "r4", "r5", "r6", "r7", "d1", "d2", "d3", "d4", "d5", "d6", "d7"]
features2 = ["c1", "c2", "c3", "c4", "c5", "c6"]



In [210]:
est_gp_c = SymbolicRegressor(population_size=400, feature_names=features2,
                           generations=20, tournament_size=3, stopping_criteria=0,
                           p_crossover=0.3, p_subtree_mutation=0.4,
                           p_hoist_mutation=0.3, p_point_mutation=0,
                           const_range=(-1,1), init_method="half and half",
                           function_set=functions,  metric="mse",
                           max_samples=0.9, verbose=1, init_depth=(2,5),
                           parsimony_coefficient=0.01, random_state=0)
est_gp_c.fit(difference_c_train.drop(["c_dif_target"], axis=1), difference_c_train["c_dif_target"])


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    12.42      2.74024e+35        6           614836           943675     42.68s
   1     9.16      2.79758e+38        4           615929           933285     35.02s
   2     7.68      2.42128e+23        6           620281           893946     29.46s
   3     6.04      8.04648e+14        7           569179      1.35423e+06     23.37s
   4     5.89      2.36705e+23        4           570090      1.34574e+06     20.06s
   5     5.03      2.71857e+23        8           586693      1.19637e+06     18.04s
   6     5.04      1.24092e+16        1           506395      1.91888e+06     17.41s
   7     5.41      1.60294e+16        5           569675      1.33621e+06     16.68s
   8     6.06      3.83809e+22        3           575139      1.30029e+06  

SymbolicRegressor(const_range=(-1, 1),
                  feature_names=['c1', 'c2', 'c3', 'c4', 'c5', 'c6'],
                  function_set=['add', 'sub', 'mul', 'div', 'inv', 'log'],
                  generations=20, init_depth=(2, 5),
                  init_method='half and half', low_memory=False,
                  max_samples=0.9, metric='mse', n_jobs=1, p_crossover=0.3,
                  p_hoist_mutation=0.3, p_point_mutation=0,
                  p_point_replace=0.05, p_subtree_mutation=0.4,
                  parsimony_coefficient=0.01, population_size=400,
                  random_state=0, stopping_criteria=0, tournament_size=3,
                  verbose=1, warm_start=False)

In [216]:
est_gp_d.score(difference_d_test.drop(["d_dif_target"], axis=1), difference_d_test["d_dif_target"])

0.8478601150547881

In [212]:
est_gp_r = SymbolicRegressor(population_size=400, feature_names=["r1", "r2", "r3","r4", "r5", "r6"],
                           generations=20, tournament_size=3, stopping_criteria=0,
                           p_crossover=0.7, p_subtree_mutation=0.2,
                           p_hoist_mutation=0.0, p_point_mutation=0,
                           const_range=(-1,1), init_method="half and half",
                           function_set=functions,  metric="mse",
                           max_samples=0.9, verbose=1, init_depth=(2,4),
                           parsimony_coefficient=0.01, random_state=0)
est_gp_r.fit(difference_r_train.drop(["r_dif_target"], axis=1), difference_r_train["r_dif_target"])

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.16      9.92918e+32       11      3.19627e+06       7.8295e+06     33.43s
   1     7.34      5.78967e+22        4      2.40033e+06      1.03974e+07     30.84s
   2     6.99      2.64632e+15       11      2.36675e+06      1.06977e+07     29.06s
   3     6.66       1.6996e+14        1      2.10566e+06      1.30474e+07     24.49s
   4     6.04      2.80977e+14       24      2.32668e+06      1.10587e+07     21.09s
   5     5.85      2.78219e+14        3       2.1147e+06      1.10386e+07     19.34s
   6     6.54      2.44086e+14        5      1.93715e+06      1.45636e+07     19.78s
   7     6.57      2.38754e+14        3      2.02773e+06      1.37487e+07     17.40s
   8     7.12      7.08331e+22        6      2.30898e+06      9.29045e+06  

SymbolicRegressor(const_range=(-1, 1),
                  feature_names=['r1', 'r2', 'r3', 'r4', 'r5', 'r6'],
                  function_set=['add', 'sub', 'mul', 'div', 'inv', 'log'],
                  generations=20, init_depth=(2, 4),
                  init_method='half and half', low_memory=False,
                  max_samples=0.9, metric='mse', n_jobs=1, p_crossover=0.7,
                  p_hoist_mutation=0.0, p_point_mutation=0,
                  p_point_replace=0.05, p_subtree_mutation=0.2,
                  parsimony_coefficient=0.01, population_size=400,
                  random_state=0, stopping_criteria=0, tournament_size=3,
                  verbose=1, warm_start=False)

In [211]:
est_gp_d = SymbolicRegressor(population_size=400, feature_names=features2,
                           generations=20, tournament_size=3, stopping_criteria=0,
                           p_crossover=0.4, p_subtree_mutation=0.2,
                           p_hoist_mutation=0.2, p_point_mutation=0,
                           const_range=(-1,1), init_method="half and half",
                           function_set=functions,  metric="mse",
                           max_samples=0.9, verbose=1, init_depth=(2,5),
                           parsimony_coefficient=0.01, random_state=0)
est_gp_d.fit(difference_d_train.drop(["d_dif_target"], axis=1), difference_d_train["d_dif_target"])

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    12.42      7.91551e+23        7          3562.72          10869.3     39.46s
   1     9.20       1.0815e+24        6          3865.38          7928.17     33.50s
   2     9.54      6.30859e+09        1          3271.74          13229.9     32.55s
   3     7.99      4.95621e+08        9          3129.76          14607.2     24.59s
   4     7.20       6.2142e+12        9          2974.28          16006.2     21.26s
   5     6.11      2.70165e+13        1          2491.68            20249     17.49s
   6     6.22      4.09814e+13        6          3026.05          15425.9     17.14s
   7     5.46       5.0984e+13        1          3040.44          15311.2     14.78s
   8     5.03      8.00678e+08        1          3133.07          14477.7  

SymbolicRegressor(const_range=(-1, 1),
                  feature_names=['c1', 'c2', 'c3', 'c4', 'c5', 'c6'],
                  function_set=['add', 'sub', 'mul', 'div', 'inv', 'log'],
                  generations=20, init_depth=(2, 5),
                  init_method='half and half', low_memory=False,
                  max_samples=0.9, metric='mse', n_jobs=1, p_crossover=0.4,
                  p_hoist_mutation=0.2, p_point_mutation=0,
                  p_point_replace=0.05, p_subtree_mutation=0.2,
                  parsimony_coefficient=0.01, population_size=400,
                  random_state=0, stopping_criteria=0, tournament_size=3,
                  verbose=1, warm_start=False)

In [207]:
print(est_gp._program)
est_gp.score

add(log(sub(div(c2, c3), inv(0.744))), c6)


<bound method RegressorMixin.score of SymbolicRegressor(const_range=(-1, 1),
                  feature_names=['c1', 'c2', 'c3', 'c4', 'c5', 'c6'],
                  function_set=['add', 'sub', 'mul', 'div', 'inv', 'log'],
                  generations=20, init_depth=(2, 5),
                  init_method='half and half', low_memory=False,
                  max_samples=0.9, metric='mse', n_jobs=1, p_crossover=0.3,
                  p_hoist_mutation=0.3, p_point_mutation=0,
                  p_point_replace=0.05, p_subtree_mutation=0.4,
                  parsimony_coefficient=0.01, population_size=400,
                  random_state=0, stopping_criteria=0, tournament_size=3,
                  verbose=1, warm_start=False)>

In [ ]:
new_deaths = deaths_data.drop(["Province/State", "Country/Region", "Lat", "Long"], axis = 1)
to_split = new_deaths.to_numpy()[2]
inp, outp =  split_sequence(to_split, 7)
len(outp)



207

In [ ]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
def work(df_c, df_r, df_d, case_type, n):
  np_c = df_c.drop(["Province/State", "Country/Region", "Lat", "Long"], axis = 1).to_numpy()[2]
  np_r = df_r.drop(["Province/State", "Country/Region", "Lat", "Long"], axis = 1).to_numpy()[2]
  np_d = df_d.drop(["Province/State", "Country/Region", "Lat", "Long"], axis = 1).to_numpy()[2]	
	
	# Run the little function
  inputs_conf, targets_conf = split_sequence(np_c, n)
  inputs_recov, targets_recov = split_sequence(np_r, n)
  inputs_death, targets_death = split_sequence(np_d, n)
	
  count = len(targets_conf)
	
  if case_type == "c":
	  targets = targets_conf
		
  elif case_type == "r":
	  targets = targets_recov
		
  else:
	  targets = targets_death

	
  print ("---------------------------------------------------------------")
  print ("The number fitness cases for location #" + str(2)+": " + str(count))
  print ("The requested targets are : " + case_type)
  print ("---------------------------------------------------------------")
	return_array = np.array()
  for x in range(count):
    # inputs = inputs_conf[x]
    inputs = np.append(inputs_conf[x], inputs_recov[x])
    inputs = np.append(inputs, inputs_death[x])
    print("inputs:")
    print(inputs)
    print("target: " + str(targets[x]) + "\n")

In [ ]:
work(confirmed_data, recovered_data, deaths_data, "d", 7)

---------------------------------------------------------------
The number fitness cases for location #2: 207
The requested targets are : d
---------------------------------------------------------------
inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
target: 0

inputs:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0